기본 설정(원자, 스캔 대역, 파장창)

In [23]:
import math, random, time
from dataclasses import dataclass
from typing import List, Tuple, Dict
import pandas as pd

from arc import Caesium, Rubidium  # ARC의 원자 클래스 (영국식: Caesium)
# 주요 API: getTransitionFrequency, getTransitionWavelength, getDipoleMatrixElement, getTransitionRate, getStateLifetime 등
# (ARC 문서 Detailed API 참조)

# ----- 사용자 설정 -----
ATOM_SPECIES = "Cs"                 # "Cs" 또는 "Rb"
F_THz_MIN, F_THz_MAX = 0.6e12, 1.0e12  # 스캔할 THz 주파수 대역 [Hz]
N_MIN, N_MAX = 10, 40               # 탐색할 n 구간
L_PAIRS = [("P","D"), ("D","P")]    # Rydberg-Rydberg 전이 후보 계열
J_OPTIONS = {"P":[1.5], "D":[2.5]}  # 빠른 스캔용 j 후보
VISIBLE_WINDOW = (500e-9, 600e-9)   # 카메라 민감 대역(500–600 nm)
ETHz_FOR_FOM = 1.0                  # FoM 비교용 정규화 E(전계) [V/m]
MC_CASCADES = 800                   # 형광 캐스케이드 몬테카를로 샷 수(속도↔정확도)
MAX_DECAY_STEPS = 8                 # 캐스케이드 최대 단계
TOPK = 20                           # 상위 몇 개 보여줄지
SEED = 7
random.seed(SEED)

# 편광(q)과 m_j 설정: 기본은 π-편광(q=0), m_j=+1/2
POL_Q = 0      # q = -1(σ-), 0(π), +1(σ+)
M_J   = 0.5    # j=3/2, 5/2 등에 대해 Δm = q 선택규칙 자동 반영


# 물리 상수(독립 사용)
HBAR = 1.054_571_817e-34  # [J·s]
C = 299_792_458.0         # [m/s]
H = 6.626_070_15e-34      # [J·s]

def make_atom(name:str):
    if name.lower() in ("cs","caesium","cesium"):
        return Caesium()
    elif name.lower() in ("rb","rubidium"):
        return Rubidium()
    raise ValueError("ATOM_SPECIES must be 'Cs' or 'Rb'")

def l_letter_to_l(ltr:str)->int:
    return {"S":0,"P":1,"D":2,"F":3,"G":4,"H":5}[ltr.upper()]

def trans_freq_hz(atom, s1, s2)->float:
    n1,l1,j1 = s1; n2,l2,j2 = s2
    return abs(atom.getTransitionFrequency(n1,l1,j1, n2,l2,j2))  # [Hz]

def trans_wavelength_m(atom, s1, s2)->float:
    n1,l1,j1 = s1; n2,l2,j2 = s2
    # ARC가 바로 파장 제공
    lam = abs(atom.getTransitionWavelength(n1,l1,j1, n2,l2,j2))  # [m]
    return lam

def omega_1Vpm(atom, s1, s2, mj1=M_J, q=POL_Q) -> float:
    """E=1 V/m에서의 라비 주파수 [rad/s] (ARC 내장 함수 사용)."""
    n1,l1,j1 = s1; n2,l2,j2 = s2
    try:
        # getRabiFrequency2(n1,l1,j1,mj1, n2,l2,j2, q, E[V/m])
        return abs(atom.getRabiFrequency2(n1,l1,j1, mj1, n2,l2,j2, q, 1.0))
    except Exception:
        return 0.0

def d_from_omega(Omega_rad_s: float, E=1.0) -> float:
    """|d| [C·m] 추정: d = ħΩ/E."""
    return HBAR * Omega_rad_s / E

def in_visible(lam:float, window=VISIBLE_WINDOW)->bool:
    return (window[0] <= lam <= window[1])


셀 3 — 형광 캐스케이드(간단 몬테카를로)

In [24]:
def allowed_lowers(n:int,l:int,j:float, n_floor:int=5):
    """E1 허용 하향 상태의 간단한 생성기 (근사)."""
    outs = []
    for lp in (l-1, l+1):
        if lp < 0: 
            continue
        for jp in (lp-0.5, lp+0.5):
            if jp <= 0: 
                continue
            # n' 영역: [n_floor .. n-1] 중 앞부분만 일부 샘플(속도용)
            nmin = max(n_floor, 5)
            nmax = max(n_floor, n-1)
            for np in range(nmin, min(nmax, nmin+14)+1):  # 최대 15개 정도만(가벼운 근사)
                outs.append((np, lp, jp))
    # 중복 제거
    uniq = []
    seen = set()
    for t in outs:
        if t not in seen:
            seen.add(t); uniq.append(t)
    return uniq

def mc_visible_yield(atom, n:int,l:int,j:float,
                     shots:int=MC_CASCADES,
                     max_steps:int=MAX_DECAY_STEPS,
                     window:Tuple[float,float]=VISIBLE_WINDOW):
    """상태 (n,l,j)에서 시작하는 자발붕괴 캐스케이드 중 500–600nm 광자 평균 개수 추정."""
    vis = 0
    for _ in range(shots):
        cur = (n,l,j)
        for _ in range(max_steps):
            lowers = allowed_lowers(*cur)
            # 전이율(A)로 가중 무작위 선택
            A_list = []
            for t in lowers:
                try:
                    A = atom.getTransitionRate(cur[0],cur[1],cur[2], t[0],t[1],t[2])  # [s^-1]
                except Exception:
                    A = 0.0
                if A>0: 
                    A_list.append((t,A))
            if not A_list:
                break
            totalA = sum(A for _,A in A_list)
            r = random.random()*totalA
            acc = 0.0
            next_state = None
            for t,A in A_list:
                acc += A
                if r <= acc:
                    next_state = t
                    break
            lam = trans_wavelength_m(atom, cur, next_state)
            if lam>0 and in_visible(lam, window):
                vis += 1
            # 종결 조건(낮은 n·낮은 l에 오면 종료)
            if next_state[0] <= 7 and next_state[1] <= 1:
                break
            cur = next_state
    return vis/float(shots)


후보 스캔 · 점수화 · 테이블 출력

In [26]:
@dataclass
class Candidate:
    upper: Tuple[int,int,float]
    lower: Tuple[int,int,float]
    f_THz: float
    d_Cm: float
    Omega_1Vpm: float
    vis_yield_best: float

def scan_candidates(atom_name="Cs") -> List[Candidate]:
    atom = make_atom(atom_name)
    out: List[Candidate] = []
    t0 = time.time()
    for l1ltr, l2ltr in L_PAIRS:
        l1 = l_letter_to_l(l1ltr); l2 = l_letter_to_l(l2ltr)
        for j1 in J_OPTIONS[l1ltr]:
            for j2 in J_OPTIONS[l2ltr]:
                for n1 in range(N_MIN, N_MAX+1):
                    for dn in (0, -1, +1, -2, +2):  # 인접/근접 상태 위주
                        n2 = n1 + dn
                        if n2 < N_MIN or n2 > N_MAX or n2 == n1:
                            continue
                        s1, s2 = (n1,l1,j1), (n2,l2,j2)
                        f = trans_freq_hz(atom, s1, s2)
                        if F_THz_MIN <= f <= F_THz_MAX:
                            d = d_from_omega(Omega, E=ETHz_FOR_FOM)          # [C·m], 표기용
                            Omega = omega_1Vpm(atom, s1, s2)                 # [rad/s] @ 1 V/m
                            # 형광 수율(상·하 상태 중 더 유리한 쪽 채택)
                            y1 = mc_visible_yield(atom, *s1)
                            y2 = mc_visible_yield(atom, *s2)
                            out.append(Candidate(upper=s1, lower=s2,
                                                 f_THz=f, d_Cm=d, Omega_1Vpm=Omega,
                                                 vis_yield_best=max(y1,y2)))
    dt = time.time()-t0
    print(f"scan done: {len(out)} candidates in {dt:.1f}s")
    return out

def score(c: Candidate)->float:
    # FoM = Ω(1V/m) × (가시 형광 수율)
    return abs(c.Omega_1Vpm) * c.vis_yield_best

def fmt_state(s):
    n,l,j = s
    LTR = "SPDFGH"[l] if l<6 else f"L{l}"
    return f"{n}{LTR}_{int(2*j)}/2"

def to_dataframe(cands: List[Candidate], topk=TOPK)->pd.DataFrame:
    rows = []
    for c in sorted(cands, key=lambda x: score(x), reverse=True)[:topk]:
        rows.append({
            "upper": fmt_state(c.upper),
            "lower": fmt_state(c.lower),
            "f_THz": c.f_THz/1e12,
            "|d| [C·m]": c.d_Cm,
            "Ω(1 V/m) [Hz]": c.Omega_1Vpm/(2*math.pi),
            "visible_yield(500–600nm)": c.vis_yield_best,
            "FoM": score(c)
        })
    return pd.DataFrame(rows)

# 실행
cands = scan_candidates(ATOM_SPECIES)
df = to_dataframe(cands, topk=TOPK)
#"Ω(1 V/m) [Hz]": Omega/(2*math.pi)

UnboundLocalError: local variable 'Omega' referenced before assignment

In [ ]:
#빠른 자체 테스트
atom = make_atom("Cs")
s1 = (14, l_letter_to_l("P"), 1.5)
s2 = (13, l_letter_to_l("D"), 2.5)
print("λ (m):", trans_wavelength_m(atom, s1, s2))
print("A_ij (s^-1):", atom.getTransitionRate(*s1, *s2))
print("Ω @1 V/m (rad/s):", omega_1Vpm(atom, s1, s2, mj1=M_J, q=POL_Q))

λ (m): 0.0005465790711620962
A_ij (s^-1): 0.0
Ω @1 V/m (rad/s): 6413605.817088714
